<a href="https://colab.research.google.com/github/davidheram/Intro-to-NLP/blob/main/intro_to_nlp_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [204]:
import pandas as pd


In [205]:
df = pd.read_csv("https://breathecode.herokuapp.com/asset/internal-link?id=932&path=url_spam.csv")

In [206]:
df

,url,is_spam
0,https://briefingday.us8.list-manage.com/unsubs...,True
1,https://www.hvper.com/,True
2,https://briefingday.com/m/v4n3i4f3,True
3,https://briefingday.com/n/20200618/m#commentform,False
4,https://briefingday.com/fan,True
...,...,...
2994,https://www.smartcitiesworld.net/news/news/dee...,False
2995,https://www.youtube.com/watch,True
2996,https://techcrunch.com/2019/07/04/an-optimisti...,False
2997,https://www.technologyreview.com/2019/12/20/13...,False


In [207]:
df["is_spam"].value_counts()

,count
is_spam,
False,2303
True,696


In [208]:
df =df.drop_duplicates()
df =df.reset_index(inplace=False, drop=True)

In [209]:
df["is_spam"].value_counts()

,count
is_spam,
False,2125
True,244


In [210]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download("stopwords")
nltk.download("WordNetLemmatizer")
nltk.download("wordnet")

stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Error loading WordNetLemmatizer: Package
[nltk_data]     'WordNetLemmatizer' not found in index
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [211]:
def preprocess_url(url):
  url = url.lower()
  tokens = re.split(r'[\/\.\-\?\=\&\_]', url)


  tokens = [lemmatizer.lemmatize(t)
  for t in tokens
            if t and not t.isdigit()and t not in stop_words]
  return " ".join(tokens)

In [212]:
df["clean_url"] = df["url"].apply(preprocess_url)

In [213]:
from numpy import vectorize
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(ngram_range=(3,5), max_features=5000)

X= vectorizer.fit_transform(df["clean_url"])
y= df["is_spam"]


In [214]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state= 134)

In [215]:
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, confusion_matrix

svm_model = LinearSVC()
svm_model = svm_model.fit(X_train, y_train)

In [216]:
y_pred = svm_model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       False       0.95      0.97      0.96       425
        True       0.69      0.55      0.61        49

    accuracy                           0.93       474
   macro avg       0.82      0.76      0.79       474
weighted avg       0.92      0.93      0.92       474

